<img align="left" style="padding-right:10px;" src="figures/cartel.jpg">
<!--COURSE_INFORMATION-->
## This notebook contains a unit from the course [Biology Meets Programming](https://www.coursera.org/learn/bioinformatics/home/welcome) by University of California in Coursera 


### The content is available [on GitHub](https://github.com/vencejo/Curso_BiologyMeetsProgramming).

<!--NAVIGATION-->
< [2.2 Asymmetry of Replication](2.2 Asymmetry of Replication.ipynb) | [Contents](Index.ipynb) | [2.4 The Skew Diagram](2.4 The Skew Diagram.ipynb)>


### Deamination

In the last section, we saw that as the replication fork expands, DNA polymerase synthesizes DNA quickly on the reverse half-strand but suffers delays on the forward half-strand. We will explore the asymmetry of DNA replication to design a new algorithm for finding ori. 

How in the world can the asymmetry of replication possibly help us locate ori? Notice that since the replication of a reverse half-strand proceeds quickly, it lives double-stranded for most of its life. Conversely, a forward half-strand spends a much larger amount of its life single-stranded, waiting to be used as a template for replication. This discrepancy between the forward and reverse half-strands is important because single-stranded DNA has a much higher mutation rate than double-stranded DNA. In particular, if one of the four nucleotides in single-stranded DNA has a greater tendency than other nucleotides to mutate in single-stranded DNA, then we should observe a shortage of this nucleotide on the forward half-strand.


Following up on this thought, let’s compare the nucleotide counts of the reverse and forward half-strands. If these counts differ substantially, then we will design an algorithm that attempts to track down these differences in genomes for which ori is unknown. The nucleotide counts for Thermotoga petrophila are shown in the figure below.

<img align="center" style="padding-right:10px;" src="figures/fig23.png">

Although the frequencies of A and T are practically identical on the two half-strands, C is more frequent on the reverse half-strand than on the forward half-strand, resulting in a difference of 219518 - 207901 = +11617. Its complementary nucleotide G is less frequent on the reverse half-strand than on the forward half-strand, resulting in a difference of 201634 - 211607 = -9973.



It turns out that we observe these discrepancies because cytosine (C) has a tendency to mutate into thymine (T) through a process called deamination. Deamination rates rise 100-fold when DNA is single-stranded, which leads to a decrease in cytosine on the forward half-strand, thus forming mismatched base pairs T-G. These mismatched pairs can further mutate into T-A pairs when the bond is repaired in the next round of replication, which accounts for the observed decrease in guanine (G) on the reverse half-strand (recall that a forward parent half-strand synthesizes a reverse daughter half-strand, and vice-versa).

STOP and Think: If deamination changes cytosine to thymine, why do you think that the forward half-strand still has some cytosine?

Let’s see if we can take advantage of these peculiar statistics caused by deamination to locate ori in a circular bacterial genome. Since we know that C is more frequent in half of the genome and less frequent in the other half, our idea is to slide a giant window of length len(Genome)//2 down the genome, counting the number of occurrences of C in each window. (Note: in Python, the double slash // indicates integer division, or eliminating any remainder; therefore, 11//2 is equal to 5, not 5.5.) Inspired by the nucleotide counts table in Vibrio cholerae (reproduced below), our hope is that the window having the fewest occurrences of C will roughly correspond to the forward half-strand and that the window having the most occurrences of C will roughly correspond to the reverse half-strand. And if we know where the forward and reverse half-strands are, then we have found ori!

Although most bacteria have circular genomes, we have thus far assumed that genomes were linear, a reasonable simplifying assumption because the length of the window is much shorter than the length of the genome. This time, because we are sliding a giant window, we should account for windows that “wrap around” the end of Genome. To do so, we will define a string ExtendedGenome as Genome+Genome[0:n//2]. That is, we copy the first len(Genome)//2 nucleotides of Genome to the end of the string (figure below).

CTGCTTCGCCCGCCGGACCGGCCTCGTGATGGGGT**CTGCTTCGCCCGCCGGA**

Figure: A DNA string Genome (shown in black) containing 35 nucleotides that is extended by its first 17 nucleotides (shown in green) to yield ExtendedGenome.


We will keep track of the total number of occurrences of C that we encounter in each window of ExtendedGenome by using a symbol array. The i-th element of the symbol array is equal to the number of occurrences of the symbol in the window of length len(Genome)//2 starting at position i of ExtendedGenome. For example, see the figure below.

<img align="center" style="padding-right:10px;" src="figures/fig24.png">



Fortunately, we already know how to count the number of occurrences of C in a window of ExtendedGenome: use the function PatternCount! We can therefore define the following function that takes strings Genome and symbol as input and returns the symbol array of Genome corresponding to symbol.

```python

# Input:  Strings Genome and symbol
# Output: SymbolArray(Genome, symbol)
def SymbolArray(Genome, symbol):
    array = {}
    n = len(Genome)
    ExtendedGenome = Genome + Genome[0:n//2]
    for i in range(n):
        array[i] = PatternCount(symbol, ExtendedGenome[i:i+(n//2)])
    return array

# Reproduce the PatternCount function on the following line from Replication.py.
def PatternCount(Pattern, Text):
    count = 0
    for i in range(len(Text)-len(Pattern)+1):
        if Text[i:i+len(Pattern)] == Pattern:
            count = count+1
    return count
```

Code Challenge (1 point): Re-type this algorithm into the code window below.

Click here for this problem's test datasets.

Sample Input:

AAAAGGGG
A

Sample Output:

{0: 4, 1: 3, 2: 2, 3: 1, 4: 0, 5: 1, 6: 2, 7: 3}



STOP and Think: Try running SymbolArray with Genome equal to the E. coli genome and symbol equal to "C". What happens? Why? (Don't try solving this exercise more than once.)

The genome will be fed into your code as a Python string, but in the event that you want to play with the E. coli genome on your own computer, you can download the genome [here](dnas/E_coli.txt).

Didn't work.

Failed test #2. Time limit exceeded

Time Limit: 15 seconds
Memory Limit: 512 MB






If you attempted the previous exercise, then you may have wondered why the time limit was exceeded. The reason why is that SymbolArray is very slow.

But wait, you may be wondering. Aren’t computers lightning-fast? Yet although computer speed has increased dramatically, even the fastest computer in the world cannot execute a program based on an inefficient algorithm.

Why is SymbolArray inefficient? Its for loop makes n = len(Genome) iterations. Then, to compute PatternCount(symbol, ExtendedGenome(i:i+(n//2))), we must compare symbol against n//2 symbols of ExtendedGenome. As a result, we require a total of n2//2 comparisons to execute SymbolArray(Genome, symbol). For a bacterial genome such as E. coli, which contains over 4.5 million nucleotides, we will need to execute over ten trillion symbol comparisons in order to generate a symbol array, which could take several days on a fast home computer operating several million comparisons per second. For more on running time analysis, see DETOUR: [Big-O notation](2.7 Detour Big-O Notation.ipynb).



### From an inefficient to an efficient algorithm

Just because SymbolArray is inefficient does not imply that a quick algorithm for constructing a symbol array does not exist. To develop a faster algorithm, we will think about what currently happens in the for loop within SymbolArray. To compute PatternCount(symbol, ExtendedGenome[0:n//2]), the algorithm consults n//2 positions of ExtendedGenome, starting at position 0. It then returns all the way back to position 1 of ExtendedGenome to compute PatternCount(symbol, ExtendedGenome[1:1+(n//2)], symbol), which consults positions 1 through 1+(n//2) of ExtendedGenome. Can we generate a symbol array using only one pass down ExtendedGenome and therefore only n+(n//2) total symbol comparisons? Such a method would reduce the running time of an algorithm generating the symbol array from a few days to a few seconds.


To speed up SymbolArray, we observe that when we slide a window one symbol to the right, the number of occurrences of symbol in the window does not change much, and so regenerating the entire array from scratch is inefficient. For example, in "CTGCTTCGCCCGCCGGACCGGCCTCGTGATGGGGTATGCTTCGCCCGCCGGA", the number of occurrences of C in the window starting at position 1 ("TGCTTCGCCCGCCGGA") can be easily computed from the number of positions of occurrences of C in the window starting at position 0 ("CTGCTTCGCCCGCCGG"). Indeed, we can view this sliding of the window as simply removing the first symbol from the window (C) and adding a new symbol to the end (A). Thus, when shifting the window right by one symbol, the number of occurrences of C in the window decreased by 1 and increased by 0. Once we compute that array[0] is equal to 8, we automatically know that array[1] is equal to 7.



This idea motivates the following algorithm, in which we only need to consider two symbols each time we slide the window.  It uses a form of range,

```python
for i in range(a, b)
```
in which we consider values of i starting with a and ending with b-1.

```python
def FasterSymbolArray(Genome, symbol):
    array = {}
    n = len(Genome)
    ExtendedGenome = Genome + Genome[0:n//2]
    array[0] = PatternCount(symbol, Genome[0:n//2])
    for i in range(1, n):
        array[i] = array[i-1]
        if ExtendedGenome[i-1] == symbol:
            array[i] = array[i]-1
        if ExtendedGenome[i+(n//2)-1] == symbol:
            array[i] = array[i]+1
    return array
```

Code Challenge (1 point): Re-type this algorithm into the code window below. Then add this function to Replication.py.

Click here for this problem's test datasets.

Sample Input:

AAAAGGGG
A

Sample Output:

{0: 4, 1: 3, 2: 2, 3: 1, 4: 0, 5: 1, 6: 2, 7: 3}



### Finding ori

The figure below visualizes the symbol array for E. coli and symbol equal to "C". Notice the clear pattern in the data! The maximum value of the array occurs around position 1600000, and the minimum value of the array occurs around position 4000000. We can therefore infer that the reverse half-strand begins around position 1600000, and that the forward half-strand begins around position 4000000. Because we know that ori occurs where the reverse half-strand transitions to the forward half-strand, we have discovered that ori is located in the neighborhood of position 4000000 of the E. coli genome, without ever needing to put on a lab coat!

<img align="center" style="padding-right:10px;" src="figures/fig25.png">

<!--NAVIGATION-->
< [2.2 Asymmetry of Replication](2.2 Asymmetry of Replication.ipynb) | [Contents](Index.ipynb) | [2.4 The Skew Diagram](2.4 The Skew Diagram.ipynb)>